In [3]:
import typing as tp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from heapq import heappush, heappop
from numpy.random import exponential
from enum import Enum

In [42]:
class EventType(Enum):
    Arrival = 1
    Service = 2
    Rejection = 3


class Event:
    def __init__(self, event_type: EventType, event_id: int, time: float, client: int) -> None:
        self.type = event_type
        self.channels_taken = 0
        self.event_id = event_id
        self.time = time
        self.client = client

    def __lt__(self, other) -> bool:
        return self.time < other.time

    def __hash__(self) -> int:
        return hash((self.event_id, self.time, self.client))

    def __str__(self) -> str:
        return f'type={self.type} id={self.event_id} time={self.time} client={self.client}'

    def __repr__(self) -> str:
        return str(self)

In [43]:
class ExponentialGenerator:
    def __init__(self, param: float) -> None:
        self.param = 1. / param

    def __iter__(self) -> 'ExponentialGenerator':
        return self

    def __next__(self) -> float:
        return exponential(self.param)

In [44]:
class EventQueue:
    def __init__(self) -> None:
        self.events: tp.Set[Event] = set()
        self.client_to_event: tp.Dict[int, Event] = {}
        self.heap: tp.List[Event] = []

    def push(self, event: Event) -> None:
        heappush(self.heap, event)
        self.client_to_event[event.client] = event
        self.events.add(event)

    def pop(self) -> tp.Optional[Event]:
        while not self.empty():
            event = heappop(self.heap)
            if event in self.events:
                self.events.remove(event)
                self.client_to_event.pop(event.client)
                return event

    def remove(self, client: int) -> None:
        self.events.remove(self.client_to_event[client])
        self.client_to_event.pop(client)

    def clear(self) -> None:
        self.heap.clear()

    def empty(self) -> bool:
        return len(self.heap) == 0

    def __len__(self) -> int:
        return len(self.heap)

    def __str__(self) -> str:
        return str(self.heap)

    def __repr__(self) -> str:
        return str(self)

In [97]:
class ChannelCoopSimulator:
    def __init__(
            self,
            channels: int,
            arrival_rate: float,
            max_request_channels: int,
            service_function: tp.Callable[[int], float]
    ) -> None:
        self.channels = channels
        assert arrival_rate > 0
        self.arrival_rate = arrival_rate
        self.arrival_generator = ExponentialGenerator(arrival_rate)
        self.service_function = service_function
        self.max_request_channels = max_request_channels
        self.event_queue = EventQueue()
        self.successes = 0
        self.states = np.zeros(channels + 1, dtype='float64')

    def __call__(self, max_time: float) -> None:
        self.event_queue.clear()
        self.event_queue.push(Event(EventType.Arrival, 0, next(self.arrival_generator), 0))
        self.states.fill(0)
        clients_count = 1
        busy_channels = 0
        events_count = 1
        current_time = 0.
        while current_time < max_time:
            event = self.event_queue.pop()
            state = busy_channels
            self.states[state] += event.time - current_time
            current_time = event.time
            if event.type is EventType.Arrival:
                if busy_channels < self.channels:
                    current_channels = min(busy_channels + self.max_request_channels, self.channels) - busy_channels
                    busy_channels += current_channels
                    new_event = Event(
                        EventType.Service,
                        events_count,
                        current_time + next(ExponentialGenerator(1 / self.service_function(current_channels))),
                        event.client
                    )
                    new_event.channels_taken = current_channels
                    self.event_queue.push(new_event)
                    events_count += 1
                    if busy_channels < self.channels:
                        self.event_queue.push(Event(
                            EventType.Arrival,
                            events_count,
                            current_time + next(self.arrival_generator),
                            clients_count
                        ))
                        events_count += 1
                        clients_count += 1
                        
                else:
                    pass
                    # raise ValueError('Something went wrong: trying to add customer with full queue')
            elif event.type is EventType.Service:
                self.event_queue.push(Event(
                    EventType.Arrival,
                    events_count,
                    current_time + next(self.arrival_generator),
                    clients_count
                ))
                events_count += 1
                clients_count += 1
                busy_channels -= event.channels_taken
    
    def evaluate(self) -> tp.Tuple[np.ndarray, int, int]:
        if np.sum(self.states) == 0:
            raise ValueError('Simulation has not been running or no events happened')
        return self.states / np.sum(self.states)

    def get_system_stats(self, probas: np.ndarray) -> tp.Dict[str, tp.Any]:
        rejection_proba = probas[-1]
        Q = 1 - rejection_proba
        A = self.arrival_rate * Q
        avg_busy_channels = self.channels * np.sum(probas[self.channels + 1:])
        return [rejection_proba, A, avg_busy_channels]

In [98]:
simulator = ChannelCoopSimulator(5, 2., 2, lambda x: x)
simulator(10000)

In [99]:
probas = simulator.evaluate()
probas

array([0.0073716 , 0.01302305, 0.07760525, 0.13836407, 0.21527971,
       0.54835632])

In [100]:
simulator.get_system_stats(probas)

[0.5483563183673222, 0.9032873632653555, 0.0]